## Cassava Leaf Disease Classification  
このコンペはキャッサバの21367枚のラベル付き画像を使って、キャッサバを４つの病気（または健康状態）に分類するコンペです。


## OverView（Summary）

### Discription（Summary）  
- キャッサバはアフリカで2番目に大きな炭水化物の供給者であり、多くの家庭農場で栽培されているが**`ウイルス性疾患が低収量の主な原因`**となっている。データサイエンスの助けを借りれば、病気を特定して治療できる可能性がある。  
- 病気を検出する既存の方法には農業専門家の助けを借りて、視覚的に検査および診断する方法がある。しかし、これは労働集約的で、供給が少なく、費用がかかるという問題がある。
- また、追加の課題として、アフリカの農家は低品質なカメラしか持っていないため、その制約のもとでうまく機能する必要がある。
- データセットは、農家が撮影し、マケレレ大学のAIラボと国立作物資源研究所（NaCRRI）が注釈をつけた**`21,367枚のラベル付き画像`**
- あなたの仕事は各キャッサバの画像を**`4つの病気のカテゴリまたは健康な葉を示す5番目のカテゴリに分類する`**こと

### Evalution
提出物は、**`分類の正確さ`**に基づいて評価されます。

### Format
コンテストの提出形式は、次の形式のcsvファイルです。

image_id,label  
1000471002.jpg,4  
1000840542.jpg,4  
etc.

### Time Line  
- 2021年2月11日 -エントリー締め切り。競技するには、この日付より前に競技規則に同意する必要があります。

- 2021年2月11日 -チーム合併の締め切り。これは、参加者がチームに参加または統合できる最後の日です。

- 2021年2月18日 -最終提出期限。

### Code Requirements
これは**`コードコンテスト`**です  
このコンテストへの提出は、**`ノートブックを通じて行う`**必要があります。  
コミット後に[コンテストに送信]ボタンをアクティブにするには、次の条件を満たす必要があります。

- CPUノートブック<= 9時間の実行時間  
- GPUノートブック<= 9時間の実行時間  
- TPUは、このコンテストへの提出には利用できません。モデルのトレーニングに引き続き使用できます。TPUでトレーニングし、GPUで推論/送信を実行する方法のウォークスルーについては、TPUドキュメントをご覧ください。
- インターネットアクセスが無効
- 事前にトレーニングされたモデルを含む、無料で公開されている外部データが許可されます
- 送信ファイルには「submission.csv」という名前を付ける必要があります

## Files  
- **train.csv**

    - image_id the image file name.

    - label the ID code for the disease.


- **sample_submission.csv**  
A properly formatted sample submission, given the disclosed test set content.

    - image_id the image file name.

    - label the predicted ID code for the disease.

## EDA

#### reference
- https://www.kaggle.com/tanlikesmath/cassava-classification-eda-fastai-starter
- https://www.kaggle.com/ihelon/cassava-leaf-disease-exploratory-data-analysis

In [ ]:
import numpy as np
import os
import pandas as pd
from fastai.vision.all import *

### Trainデータの表示

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')
os.listdir(dataset_path)

In [ ]:
train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

### Labelの表示

In [ ]:
with open(os.path.join(dataset_path, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    classes=[]
    for k, v in map_classes.items():        
        classes.append([int(k), v])
        
classes_df = pd.DataFrame(classes, columns=['Label', 'Label Details'])
print(classes_df.to_string(index=False))

#### 写真を見る

In [ ]:
train_0_df = train_df[train_df['label']==0]
train_1_df = train_df[train_df['label']==1]
train_2_df = train_df[train_df['label']==2]
train_3_df = train_df[train_df['label']==3]
train_4_df = train_df[train_df['label']==4]

In [ ]:
picture_path = Path('../input/cassava-leaf-disease-classification/train_images/')

In [ ]:
from PIL import Image
image_0 = Image.open(picture_path/train_0_df.loc[train_0_df.index[0]]['image_id'])
image_1 = Image.open(picture_path/train_1_df.loc[train_1_df.index[0]]['image_id'])
image_2 = Image.open(picture_path/train_2_df.loc[train_2_df.index[0]]['image_id'])
image_3 = Image.open(picture_path/train_3_df.loc[train_3_df.index[0]]['image_id'])
image_4 = Image.open(picture_path/train_4_df.loc[train_4_df.index[0]]['image_id'])

In [ ]:
image_0 #"Cassava Bacterial Blight (CBB)"

In [ ]:
image_1 #"Cassava Brown Streak Disease (CBSD)"

In [ ]:
image_2 #"Cassava Green Mottle (CGM)"

In [ ]:
image_3 #"Cassava Mosaic Disease (CMD)"

In [ ]:
image_4 #"Healthy"

#### クラスの件数

In [ ]:
print(f"Cassava Bacterial Blight (CBB): {len(train_0_df)}")
print(f"Cassava Brown Streak Disease (CBSD): {len(train_1_df)}")
print(f"Cassava Green Mottle (CGM): {len(train_2_df)}")
print(f"Cassava Mosaic Disease (CMD): {len(train_3_df)}")
print(f"Healthy: {len(train_4_df)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
label = ["CBB", "CBSD", "CGM", "CMD", "Healthy"]
numbers = np.array([len(train_0_df), 
                   len(train_1_df), 
                   len(train_2_df), 
                   len(train_3_df), 
                   len(train_4_df)]
                 )

plt.bar(label, numbers)

## 画像の読み込み

用意された画像はjpg形式のため、モデルに通すには形式を変換する必要があります。

#### reference  
https://www.tensorflow.org/tutorials/load_data/images?hl=ja

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

### 準備

#### 画像のパス

In [ ]:
all_image_paths=[]
for i in range(len(train_df)):
    all_image_paths.append(str(picture_path/train_df.loc[i]['image_id']))

In [ ]:
all_image_paths[:10]

#### 各画像のラベルを抜き出す

In [ ]:
all_image_labels = np.array(train_df['label'])

print("First 10 labels indices: ", all_image_labels[:10])

### 画像の読み込みと整形

一枚の画像を使って流れを確認します。

In [ ]:
img_path = all_image_paths[0]

In [ ]:
Image.open(img_path)

生の画像を取り込みます。

In [ ]:
img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")

画像のテンソルにデコードします。

In [ ]:
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

モデルに合わせてリサイズします。

In [ ]:
img_final = tf.image.resize(img_tensor, [64, 64])
img_final = img_final/255.0
print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())

このあと複数の画像をまとめて処理するので、簡単な関数にまとめます。

In [ ]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [64, 64])
    image /= 255.0  # normalize to [0,1] range

    return image

画像の表示

In [ ]:
import IPython.display as display
import pathlib
import matplotlib.pyplot as plt

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

image_path = all_image_paths[0]
label = all_image_labels[0]

plt.imshow(load_and_preprocess_image(img_path))
plt.grid(False)
plt.show()

### tf.data.Datasetの構築  
tf.data.Dataset を構築するもっとも簡単な方法は、from_tensor_slices メソッドを使うことです。

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
print(path_ds)

#### Training用の画像データセット

In [ ]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
for image in image_ds.take(1):
    new_image=image.numpy()
    print(new_image.shape)
    print(new_image)

tf.data.DatasetはそのままKerasで使うことができるようですが、やり方がわからないのでnumpyに変換します。

In [ ]:
new_image=[]
for image in image_ds:#.take(1)
    new_image.append(image.numpy())
X = np.array(new_image)
print(X.shape)

In [ ]:
import matplotlib.pyplot as plt

for n,image in enumerate(image_ds.take(4)):
    plt.figure(figsize=(8,8))
    plt.subplot(2,2,n+1)
    plt.imshow(image)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.show()

#### Labelのデータセット

In [ ]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

In [ ]:
for label in label_ds.take(10):
    print(label.numpy())

numpyに変換します。

In [ ]:
new_label=[]
for label in label_ds:#.take(1)
    new_label.append(label.numpy())
y = np.array(new_label)
print(y.shape)

## 訓練

### One Hot Encording

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
print(y.shape)
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])

print(y.shape)
print(y_one_hot.shape)

### Train_Test_Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_one_hot, test_size=0.2)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

### Kerasで訓練する

#### reference  
https://www.kaggle.com/bugraokcu/cnn-with-keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

num_classes = 5

#input image dimensions
img_rows, img_cols = 64, 64
input_shape = (img_rows, img_cols, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(384, activation='relu'))#128
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),#lr=0.01,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    batch_size=32,# 20, 32, 64
                    epochs=100,# 50
                    validation_data=(X_val,y_val))

In [ ]:
# データを確認することができる
#print(history.history)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


#### 考察
学習率0.001、バッチサイズ32くらいで学習させるとtrainデータに対しては精度が上がる。  
しかし、いくら試してもtestデータに対しての精度が上がらず、汎化性能は低い。

### 推定

In [ ]:
y_pred = model.predict(X_val)#, batch_size=1, verbose=0
print(f"y_pred.shape:{y_pred.shape}")

y_pred_label = np.argmax(y_pred, axis=1)
print(f"y_pred_label:\n{y_pred_label}")

### サブミットファイルの作成

#### テストデータの準備

In [ ]:
picture_path = Path('../input/cassava-leaf-disease-classification/test_images/')
os.listdir(picture_path)
# 一つしかない

Submitファイルの準備

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df.head()

In [ ]:
#all_image_paths_test=[]
#path_tmp = str(picture_path/sample_df.loc[0]['image_id'])
#all_image_paths_test.append(path_tmp)
#
#print(all_image_paths_test)

In [ ]:
all_image_paths_test=[]
for i in range(len(sample_df)):
    all_image_paths_test.append(str(picture_path/sample_df.loc[i]['image_id']))

In [ ]:
path_ds_test = tf.data.Dataset.from_tensor_slices(all_image_paths_test)
print(path_ds_test)

image_ds_test = path_ds_test.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
print(image_ds_test)

In [ ]:
new_image_test=[]
for image in image_ds_test:#.take(1)
    new_image_test.append(image.numpy())
X_test = np.array(new_image_test)
print(X_test.shape)


In [ ]:
y_pred_test = model.predict(X_test)#, batch_size=1, verbose=0
print(f"y_pred_test.shape:{y_pred_test.shape}")

y_pred_label_test = np.argmax(y_pred_test, axis=1)
print(f"y_pred_label_test:\n{y_pred_label_test}")

### Submission

In [ ]:
# Submission dataframe
submit_ID = sample_df.loc[:]['image_id']
submit_TARGET = pd.DataFrame(y_pred_label_test)

submission_df = pd.concat([submit_ID, submit_TARGET], axis=1)
submission_df.columns = ['image_id','label']

submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)

### Next Action  
過学習させないための工夫を検討する  
- 写真データを拡張したらどうなるか
- 特徴がわかりやすい画像を選んだり、ノイズになりそうな画像を除外して学習させたらどうなるか
- 枚数の差を調整したらどうなるか
- 画像の該当箇所だけ抜き出して学習させる方法はないか

以上